# Bert를 사용한 문장 간 관계수치 예측

2개의 문장 간의 관계를 수치로 에측한다.

0 : 무관
5 : 상관

# 필요 라이브러리 설치

In [1]:
!pip install transformers==3.0.2
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 769 kB 12.7 MB/s 
     |████████████████████████████████| 880 kB 49.6 MB/s 
     |████████████████████████████████| 1.2 MB 75.9 MB/s 
     |████████████████████████████████| 3.0 MB 79.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=f2cb0aab6138fb3ee68a65b833ce46e5b78f19918a5e5950f726f87896295399
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from transformers import BertTokenizer
from transformers import TFBertModel

import tensorflow as tf

In [3]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

SEQ_LENGTH = 128
BERT_MODEL_NAME = 'bert-base-multilingual-cased'

# 데이터

## 데이터 다운로드

In [4]:
!git clone https://github.com/kakaobrain/KorNLUDatasets

Cloning into 'KorNLUDatasets'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 1), reused 16 (delta 1), pack-reused 0
Unpacking objects: 100% (16/16), done.
Checking out files: 100% (9/9), done.


In [5]:
!wc ./KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv

  550153  8595590 78486224 ./KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv


## 데이터 로딩

In [6]:
df = pd.read_csv("KorNLUDatasets/KorSTS/sts-train.tsv", delimiter = '\t', quoting = 3)

In [7]:
df.head(10)

,genre,filename,year,id,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,1,5.00,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.
1,main-captions,MSRvid,2012test,4,3.80,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.
2,main-captions,MSRvid,2012test,5,3.80,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.
3,main-captions,MSRvid,2012test,6,2.60,세 남자가 체스를 하고 있다.,두 남자가 체스를 하고 있다.
4,main-captions,MSRvid,2012test,9,4.25,한 남자가 첼로를 연주하고 있다.,자리에 앉은 남자가 첼로를 연주하고 있다.
5,main-captions,MSRvid,2012test,11,4.25,몇몇 남자들이 싸우고 있다.,두 남자가 싸우고 있다.
6,main-captions,MSRvid,2012test,12,0.50,남자가 담배를 피우고 있다.,남자가 스케이트를 타고 있다.
7,main-captions,MSRvid,2012test,13,1.60,남자가 피아노를 치고 있다.,남자가 기타를 연주하고 있다.
8,main-captions,MSRvid,2012test,14,2.20,한 남자가 기타를 치고 노래를 부르고 있다.,한 여성이 어쿠스틱 기타를 연주하고 노래를 부르고 있다.
9,main-captions,MSRvid,2012test,16,5.00,사람이 고양이를 천장에 던지고 있다.,사람이 고양이를 천장에 던진다.


## 데이터 섞기

In [8]:
df = df.sample(frac=1).reset_index(drop=True) 

df.head()

,genre,filename,year,id,score,sentence1,sentence2
0,main-forum,deft-forum,2014,224,3.60,그들이 숨기려고 하는 것은 무엇인가?,그리고 나서 나는 물었다: 그들이 무엇을 숨기려고 하는가?
1,main-news,MSRpar,2012test,612,3.60,월스트리트가 북미 최대 정전사태 이후 다시 일어서기 위해 노력하면서 금요일 미국 회...,월스트리트가 북미에서 사상 최대의 정전사태 이후 재편되면서 미국의 주가는 금요일 깃...
2,main-news,MSRpar,2012train,257,3.75,100개국 이상의 농업부 장관들이 미국 농무부가 후원하는 3일간의 장관회의와 농업과...,앤 베네만 미국 농무장관이 월요일 3일간의 장관회의와 농업과학기술 박람회를 시작한다.
3,main-news,headlines,2013,286,0.00,미국 항공사와 조종사들은 새로운 계약에 동의한다,롬니는 이란 위협에 대해 이스라엘과 함께 서겠다고 약속한다.
4,main-forum,deft-forum,2014,344,3.80,나는 언론의 자유가 어떻게 금지될지 이해할 수 없다.,당신의 언론의 자유는 오늘날보다 더 금지되지 않을 것이다.


## 필요 입출력 값 준비

In [12]:
sentences1 = df.sentence1.values.copy().astype(str)
sentences2 = df.sentence2.values.copy().astype(str)
labels = df.score.values.copy().astype(float)/5.0

In [ ]:
print(sentences1.shape)
print(sentences2.shape)
print(labels.shape)

(5749,)
(5749,)
(5749,)


필요 시, 실습 시간 관계로 전체 중에 일부 만 사용한다.

In [ ]:
COUNT = 10000
sentences1 = sentences1[:COUNT]
sentences2 = sentences2[:COUNT]
labels = labels[:COUNT]

## 토큰나이저 생성

In [ ]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME, do_lower_case=False, model_max_length=SEQ_LENGTH)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

In [ ]:
encoded_tokens= tokenizer.encode("하늘이 푸르다.", text_pair="파란색이 좋아.")
print(encoded_tokens)
print(tokenizer.convert_ids_to_tokens(encoded_tokens))

[101, 9952, 118762, 10739, 9935, 31401, 11903, 119, 102, 9901, 49919, 41442, 10739, 9685, 16985, 119, 102]
['[CLS]', '하', '##늘', '##이', '푸', '##르', '##다', '.', '[SEP]', '파', '##란', '##색', '##이', '좋', '##아', '.', '[SEP]']


In [ ]:
tokenized = tokenizer("하늘이 푸르다.", text_pair="파란색이 좋아.", max_length=20, padding='max_length')
print(tokenizer.decode(tokenized['input_ids']))
print(tokenizer.convert_ids_to_tokens(tokenized['input_ids']))
print(tokenized['input_ids'])
print(tokenized['attention_mask'])
print(tokenized['token_type_ids'])

[CLS] 하늘이 푸르다. [SEP] 파란색이 좋아. [SEP] [PAD] [PAD] [PAD]
['[CLS]', '하', '##늘', '##이', '푸', '##르', '##다', '.', '[SEP]', '파', '##란', '##색', '##이', '좋', '##아', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]']
[101, 9952, 118762, 10739, 9935, 31401, 11903, 119, 102, 9901, 49919, 41442, 10739, 9685, 16985, 119, 102, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]


## x, y 생성


tokernizer 사용 중에 경고 메시지가 많이 뜬다. 억제한다.


In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
def build_model_input(sentences1, sentences2):
  input_ids = []
  attention_masks = []
  token_type_ids = []

  for sentence1, sentence2 in zip(sentences1, sentences2):
    tokenized = tokenizer(sentence1, text_pair=sentence2, max_length=SEQ_LENGTH, padding='max_length')
    # tokenized = {'input_ids': [101, ...], 'token_type_ids': [0, ...], 'attention_mask': [1, ...]}
    input_ids.append(tokenized['input_ids'][:SEQ_LENGTH]) # 버그인지 몰라도 SEQ_LENGTH이상이어도 더 크게 나온다.
    attention_masks.append(tokenized['attention_mask'][:SEQ_LENGTH])
    token_type_ids.append(tokenized['token_type_ids'][:SEQ_LENGTH])

  return (np.array(input_ids), np.array(attention_masks), np.array(token_type_ids))


In [ ]:
x = build_model_input(sentences1, sentences2)
y = labels

In [ ]:
print(x[0].shape)

(5749, 128)


## train/test 분리

In [ ]:
def split_bert_data(x, y, test_ratio):
  split_index = int(len(y)*(1-test_ratio))
  train_x = (x[0][:split_index], x[1][:split_index], x[2][:split_index])
  test_x  = (x[0][split_index:], x[1][split_index:], x[2][split_index:])
  train_y, test_y = y[:split_index], y[split_index:]

  return (train_x, train_y), (test_x, test_y)

(train_x, train_y), (test_x, test_y) = split_bert_data(x, y, test_ratio=0.2)

# 학습

## 모델 생성

In [ ]:
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.layers import Dense, Dropout

class TFBertPredictor(tf.keras.Model):
  def __init__(self):
    super(TFBertPredictor, self).__init__()

    self.bert = TFBertModel.from_pretrained(BERT_MODEL_NAME)
    self.dropout = Dropout(self.bert.config.hidden_dropout_prob)
    self.predcitor = Dense(1, kernel_initializer=TruncatedNormal(self.bert.config.initializer_range))

  def call(self, inputs, attention_mask=None, token_type_ids=None, training=True):

    outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
    # outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
    pooled_output = outputs[1] 
    v = self.dropout(pooled_output, training=training)
    out = self.predcitor(v)

    return out

model = TFBertPredictor()


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

참고로 Bert의 default 설정은 다음과 같다.

In [ ]:
print(model.bert.config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 119547
}



In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy


optimizer = Adam(3e-5)
loss = SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])


## 학습 실행

In [ ]:
history = model.fit(train_x, train_y, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
130/130 [==============================] - 57s 311ms/step - loss: 0.0549 - mae: 0.1870 - val_loss: 0.0322 - val_mae: 0.1361
Epoch 2/5
130/130 [==============================] - 38s 289ms/step - loss: 0.0360 - mae: 0.1493 - val_loss: 0.0317 - val_mae: 0.1349
Epoch 3/5
130/130 [==============================] - 38s 290ms/step - loss: 0.0264 - mae: 0.1282 - val_loss: 0.0302 - val_mae: 0.1318
Epoch 4/5
130/130 [==============================] - 38s 290ms/step - loss: 0.0209 - mae: 0.1142 - val_loss: 0.0340 - val_mae: 0.1378
Epoch 5/5
130/130 [==============================] - 38s 290ms/step - loss: 0.0194 - mae: 0.1102 - val_loss: 0.0324 - val_mae: 0.1301


In [ ]:
loss, mape = model.evaluate(test_x, test_y, batch_size=32)
print("loss =", loss)
print("mape =", mape)

36/36 [==============================] - 4s 99ms/step - loss: 0.0309 - mae: 0.1315
loss = 0.030873917043209076
mape = 0.13150601089000702


## 예측 실행

In [ ]:
def do_classify(sentence1, sentence2):
  model_input = build_model_input([sentence1], [sentence2])
  y_ = model.predict(model_input)
  print(sentence1, sentence2, "-->", "score :",y_[0]*5.0)

do_classify("나는 왜 그런지 잘 모르겠다.", "하늘이 푸르다.")
do_classify("나는 왜 그런지 잘 모르겠다.", "나는 왜 그런 일이 일어났는지 모르겠어.")
do_classify("나는 왜 그런지 잘 모르겠다.", "나는 왜 그런지 완전히 모르겠어.")

나는 왜 그런지 잘 모르겠다. 하늘이 푸르다. --> score : [0.20357704]
나는 왜 그런지 잘 모르겠다. 나는 왜 그런 일이 일어났는지 모르겠어. --> score : [3.731895]
나는 왜 그런지 잘 모르겠다. 나는 왜 그런지 완전히 모르겠어. --> score : [4.1359825]
